In [4]:
import os
import requests
from datetime import date, datetime, timedelta
import pandas as pd
import wikipedia
from fuzzywuzzy import fuzz, process

## Getting the Data
### Benchmark Data

First, we check to see if recent (less than a week old) UserBenchmark data exists in the directory. If not, we download the latest csv files from source.

In [25]:
parts = ['CPU', 'GPU', 'SSD', 'RAM']

def get_filename(part):
    return f'{part}_UserBenchmarks.csv'

def file_is_n_days_old(path, n):
    download = False
    
    if os.path.exists(path):
        timestamp = os.path.getmtime(path)
        modified = datetime.fromtimestamp(timestamp)
        delta = datetime.today() - modified
        if delta > timedelta(days = 7):
            download = True
    else:
        download = True
    return download

def update_bench(parts):
    fol = 'data'
    try:
        os.mkdir(fol)
    except:
        pass
    
#     os.chdir('data')
    
    for part in parts:
        filename = get_filename(part)
        path = os.path.join(fol, filename)
        
        download = file_is_n_days_old(path, 7)
            
        if download:
            url = f'https://www.userbenchmark.com/resources/download/csv/{filename}'
            response = requests.get(url)
            with open(path, 'wb') as file:
                file.write(response.content)
#     os.chdir('..')
update_bench(parts)

### CPU Data
It is difficult to find up-to-date csv files or easily accessible / correctly formatted tables for currently relevant CPU specs. The closest such resource was [CPU DB](http://cpudb.stanford.edu/), a project by Stanford. However, the database seems to be unmaintained as of 2016 at the latest. So, we will try scraping tables from Wikipedia articles instead.
#### 1. Intel

In [31]:
def update_intel(n):
    
    fol = 'data'
    filename = 'intel_cpus.csv'
    path = os.path.join(fol, filename)
    
    download = file_is_n_days_old(path, 7)
    
    if download:
    
        intel_wiki = wikipedia.page("Intel microprocessors")
        # only take most recent processors
        intel_tables = pd.read_html(intel_wiki.html())[:n]

        # filter out non-desktop processors
        intel_cpus = pd.concat([table for table in intel_tables if 'Socket' in table.columns and table['Socket'].notna().all()])
        columns = list(intel_cpus.columns)

        # consolidate column names that are diff by a couple chars
        # necessary because of weird spacing in different tables
        for i, col1 in enumerate(columns[:-1]):
            cols = columns[i+1:]
            col2, score = process.extractOne(col1, cols)
            if score > 90:
                intel_cpus[col1] = intel_cpus[col1].fillna(intel_cpus[col2])
                intel_cpus.drop(col2, axis = 1, inplace = True)
        intel_cpus.to_csv(path, encoding='utf-8', index=False)

# get top (latest) 5 tables from Wikipedia page for Intel CPUs
# skips if csv less than a week old already exists
update_intel(5)

intel_cpus = pd.read_csv('data/intel_cpus.csv')
intel_cpus.sample(5)

,Model,Price (USD),Cores/Threads,Base frequency (GHz),Max turbo frequency (GHz),GPU,Maximum GPU clock rate (MHz),L3 cache (MB),TDP (W),Socket,Release
43,G4560,$64,2/4,3.5,NaN,HD 610,1050.0,3.0,54,LGA 1151,Q1 2017
45,G3950,$52,2/2,3.0,NaN,HD 610,1050.0,2.0,51,LGA 1151,Q1 2017
24,i5-7640X,$242,4/4,4.0,4.2,NaN,NaN,6.0,112,LGA 2066,Q1 2017
0,i9-9900K,$529,8/16,3.6,5.0,UHD 630,1200.0,16.0,95,LGA 1151,Q4 2018
30,i5-7400,$182,4/4,3.0,3.5,HD 630,1000.0,6.0,65,LGA 1151,Q1 2017


#### 2. AMD